In [1]:
import os
from pathlib import Path

NOTEBOOK_PATH: Path = Path(os.path.abspath(''))
PATH_FOR_RESULT: Path = NOTEBOOK_PATH.parent.joinpath('RESULT')

# 处理压缩文件

到目前为止，我们已经从四个期货交易所（中金所、上期所、郑商所和大商所）获取了交易所提供的全部历史数据了。这其中大部分文件都是 zip 压缩文件。我们希望能够进一步的采用程序化方式，解压文件、读取数据、并存入数据库中，为以后的使用提供便利。比如，螺纹钢品种在历史上的主力合约，大致上能知道是1、5、10合约，那么具体某一年的01合约，从哪天到哪天是主力合约呢？

处理 zip 文件，Python 自带了 zipfile 模块。

In [2]:
from typing import List
import zipfile

def unzip_file(zipped_file: Path, unzip_path: Path, keep: bool = False) -> List[Path]:
    """
    解压单个 zip 文件。
    Unzip a zip file to ten temporary directory defined in <CONFIGS>, and return the unzipped file path.
    :param zipped_file:   Path，待解压文件的路径。
    :param unzip_path: Path，保存被解压出来的文件的路径。
    :param keep:       bool，保留待解压文件的话，取值 True，否则解压完成后删除待解压文件。
    :return: list，被解压出来的文件的路径（Path）的列表。
    """
    
    # 如果参数 <zip_file> 不存在，引发异常。
    if not zipped_file.exists():
        raise FileNotFoundError(f'<{zipped_file}> 不存在。')

    # 如果参数 <unzip_path> 不存在，引发异常。
    if not unzip_path.exists():
        raise FileNotFoundError(f'<{unzip_path}> 不存在。')

    # 用 zipfile 模块的 ZipFile 打开待解压文件。
    zip_file = zipfile.ZipFile(zipped_file, 'r')
    
    # 生成解压文件列表
    result: List[Path] = [unzip_path.joinpath(filename) for filename in zip_file.namelist()]
    
    # 解压文件。
    zip_file.extractall(unzip_path)
    
    # 删除待解压文件
    if not keep:
        zipped_file.unlink()
    
    # 返回解压出来的文件列表
    return result

# 测试

# 测试用待解压文件
# 如果你的 RESULT\ 文件夹没有 SHFE_2020.zip 文件，去上期所下载一个。
# 其实只要是个 zip 文件就行，可能需要修改你的 zip 文件的名字，或者下面代码中的文件名。
test_zipped_file: Path = PATH_FOR_RESULT.joinpath('SHFE_2020.zip')

# 测试用解压路径
test_unzip_path: Path = PATH_FOR_RESULT.joinpath('temp')
# 如果解压路径不存在，创建它。
if not test_unzip_path.exists():
    test_unzip_path.mkdir()

# 测试
unzip_file(
    zipped_file=test_zipped_file,
    unzip_path=test_unzip_path,
    keep=True
)

[WindowsPath('d:/Development/Jupyter/InvestmentNotebook/RESULT/temp/╦∙─┌║╧╘╝╨╨╟Θ▒¿▒φ2020.1.xls'),
 WindowsPath('d:/Development/Jupyter/InvestmentNotebook/RESULT/temp/╦∙─┌║╧╘╝╨╨╟Θ▒¿▒φ2020.2.xls'),
 WindowsPath('d:/Development/Jupyter/InvestmentNotebook/RESULT/temp/╦∙─┌║╧╘╝╨╨╟Θ▒¿▒φ2020.3.xls'),
 WindowsPath('d:/Development/Jupyter/InvestmentNotebook/RESULT/temp/╦∙─┌║╧╘╝╨╨╟Θ▒¿▒φ2020.4.xls'),
 WindowsPath('d:/Development/Jupyter/InvestmentNotebook/RESULT/temp/╦∙─┌║╧╘╝╨╨╟Θ▒¿▒φ2020.5.xls'),
 WindowsPath('d:/Development/Jupyter/InvestmentNotebook/RESULT/temp/╦∙─┌║╧╘╝╨╨╟Θ▒¿▒φ2020.6.xls'),
 WindowsPath('d:/Development/Jupyter/InvestmentNotebook/RESULT/temp/╦∙─┌║╧╘╝╨╨╟Θ▒¿▒φ2020.7.xls'),
 WindowsPath('d:/Development/Jupyter/InvestmentNotebook/RESULT/temp/╦∙─┌║╧╘╝╨╨╟Θ▒¿▒φ2020.8.xls'),
 WindowsPath('d:/Development/Jupyter/InvestmentNotebook/RESULT/temp/╦∙─┌║╧╘╝╨╨╟Θ▒¿▒φ2020.9.xls'),
 WindowsPath('d:/Development/Jupyter/InvestmentNotebook/RESULT/temp/╦∙─┌║╧╘╝╨╨╟Θ▒¿▒φ2020.10.xls'),
 WindowsPath('d:/De

震惊！这一摊是啥！！！

好吧，但凡遇到乱码问题，基本都是上古时期遗留下来的。根据维基百科，zip 格式发布于 1989 年 1 月。现行的 Unicode （通用字符集）在 1991 年 10 月发布第一版，支持中日韩字符的第二版于 1992 年 6 月发布。

因此 zip 文件的编码应该是 MS-DOS 编码，或者称为 IBM437。在 Python 中命名为 [cp437](https://docs.python.org/zh-cn/3/library/codecs.html#standard-encodings)（Code Page 437，代码页 437）。我们需要做一个转换。

In [3]:
def unzip_file(zipped_file: Path, unzip_path: Path, keep: bool = False) -> List[Path]:
    """
    解压单个 zip 文件。
    :param zipped_file:   Path，待解压文件的路径。
    :param unzip_path: Path，保存被解压出来的文件的路径。
    :param keep:       bool，保留待解压文件的话，取值 True，否则解压完成后删除待解压文件。
    :return: list，被解压出来的文件的路径（Path）的列表。
    """
    
    # 如果参数 <zip_file> 不存在，引发异常。
    if not zipped_file.exists():
        raise FileNotFoundError(f'<{zipped_file}> 不存在。')

    # 如果参数 <unzip_path> 不存在，引发异常。
    if not unzip_path.exists():
        raise FileNotFoundError(f'<{unzip_path}> 不存在。')

    # 用 zipfile 模块的 ZipFile 打开待解压文件。
    zip_file = zipfile.ZipFile(zipped_file, 'r')
    
    # 生成解压文件列表，这里的 filename 编码是不正确的。
    origin: List[Path] = [unzip_path.joinpath(filename) for filename in zip_file.namelist()]
    
    # 编码正确的文件列表
    result: List[Path] = [unzip_path.joinpath(filename.encode('cp437').decode('gbk')) for filename in zip_file.namelist()]
    
    # 解压文件。
    zip_file.extractall(unzip_path)
    
    # 重命名解压出来的文件。
    for i in range(len(origin)):
        origin[i].rename(result[i])
    
    # 删除待解压文件
    if not keep:
        zipped_file.unlink()
    
    # 返回解压出来的文件列表
    return result

# 测试

# 换一个测试用的待解压文件。
test_zipped_file: Path = PATH_FOR_RESULT.joinpath('SHFE_2019.zip')

# 测试用解压路径
test_unzip_path: Path = PATH_FOR_RESULT.joinpath('temp')
# 如果解压路径不存在，创建它。
if not test_unzip_path.exists():
    test_unzip_path.mkdir()

# 测试
unzip_file(
    zipped_file=test_zipped_file,
    unzip_path=test_unzip_path,
    keep=True
)

[WindowsPath('d:/Development/Jupyter/InvestmentNotebook/RESULT/temp/所内合约行情报表2019.3.xls'),
 WindowsPath('d:/Development/Jupyter/InvestmentNotebook/RESULT/temp/所内合约行情报表2019.4.xls'),
 WindowsPath('d:/Development/Jupyter/InvestmentNotebook/RESULT/temp/所内合约行情报表2019.5.xls'),
 WindowsPath('d:/Development/Jupyter/InvestmentNotebook/RESULT/temp/所内合约行情报表2019.1.xls'),
 WindowsPath('d:/Development/Jupyter/InvestmentNotebook/RESULT/temp/所内合约行情报表2019.2.xls'),
 WindowsPath('d:/Development/Jupyter/InvestmentNotebook/RESULT/temp/所内合约行情报表2019.6.xls'),
 WindowsPath('d:/Development/Jupyter/InvestmentNotebook/RESULT/temp/所内合约行情报表2019.7.xls'),
 WindowsPath('d:/Development/Jupyter/InvestmentNotebook/RESULT/temp/所内合约行情报表2019.8.xls'),
 WindowsPath('d:/Development/Jupyter/InvestmentNotebook/RESULT/temp/所内合约行情报表2019.9.xls'),
 WindowsPath('d:/Development/Jupyter/InvestmentNotebook/RESULT/temp/所内合约行情报表2019.10.xls'),
 WindowsPath('d:/Development/Jupyter/InvestmentNotebook/RESULT/temp/所内合约行情报表2019.11.xls'),
 Windows

哦耶！

接下来就有两种使用场景了：

- 逐交易所逐年下载，每完成一个压缩文件就解压；

- 逐交易所逐年下载，全部完成再解压。

我们是有理想的二把刀程序员，我们要为用户提供最大的可能性！

先来一个搜索所有压缩文件的程序：

In [4]:
def search_all_zip_file(directory: Path) -> List[Path]:
    """
    搜索指定目录下的所有 zip 文件。
    :param directory:  Path，待搜索的路径。
    :return: list，指定目录下的所有 zip 文件的路径（Path）的列表。
    """
    result: List[Path] = []
    for file in directory.iterdir():
        if file.suffix == '.zip':
            result.append(file)
    return result

# 测试
for item in search_all_zip_file(PATH_FOR_RESULT):
    print(item)

d:\Development\Jupyter\InvestmentNotebook\RESULT\CFFEX_2010-04.zip
d:\Development\Jupyter\InvestmentNotebook\RESULT\CFFEX_2010-05.zip
d:\Development\Jupyter\InvestmentNotebook\RESULT\CFFEX_2010-06.zip
d:\Development\Jupyter\InvestmentNotebook\RESULT\CFFEX_2010-07.zip
d:\Development\Jupyter\InvestmentNotebook\RESULT\CFFEX_2010-08.zip
d:\Development\Jupyter\InvestmentNotebook\RESULT\CFFEX_2010-09.zip
d:\Development\Jupyter\InvestmentNotebook\RESULT\CFFEX_2010-10.zip
d:\Development\Jupyter\InvestmentNotebook\RESULT\CFFEX_2010-11.zip
d:\Development\Jupyter\InvestmentNotebook\RESULT\CFFEX_2010-12.zip
d:\Development\Jupyter\InvestmentNotebook\RESULT\CFFEX_2011-01.zip
d:\Development\Jupyter\InvestmentNotebook\RESULT\CFFEX_2011-02.zip
d:\Development\Jupyter\InvestmentNotebook\RESULT\CFFEX_2011-03.zip
d:\Development\Jupyter\InvestmentNotebook\RESULT\CFFEX_2011-04.zip
d:\Development\Jupyter\InvestmentNotebook\RESULT\CFFEX_2011-05.zip
d:\Development\Jupyter\InvestmentNotebook\RESULT\CFFEX_2011-06

解压所有 zip 文件:

In [5]:
# 换一个解压路径。
another_unzip_path: Path = PATH_FOR_RESULT.joinpath('temp2')

# 如果新的解压路径不存在，创建它。
if not another_unzip_path.exists():
    another_unzip_path.mkdir()

# 解压
for item in search_all_zip_file(PATH_FOR_RESULT):
    print(f'正在解压缩 {item} ...')
    unzip_file(
        zipped_file=item,
        unzip_path=another_unzip_path,
        keep=True
    )

正在解压缩 d:\Development\Jupyter\InvestmentNotebook\RESULT\CFFEX_2010-04.zip ...
正在解压缩 d:\Development\Jupyter\InvestmentNotebook\RESULT\CFFEX_2010-05.zip ...
正在解压缩 d:\Development\Jupyter\InvestmentNotebook\RESULT\CFFEX_2010-06.zip ...
正在解压缩 d:\Development\Jupyter\InvestmentNotebook\RESULT\CFFEX_2010-07.zip ...
正在解压缩 d:\Development\Jupyter\InvestmentNotebook\RESULT\CFFEX_2010-08.zip ...
正在解压缩 d:\Development\Jupyter\InvestmentNotebook\RESULT\CFFEX_2010-09.zip ...
正在解压缩 d:\Development\Jupyter\InvestmentNotebook\RESULT\CFFEX_2010-10.zip ...
正在解压缩 d:\Development\Jupyter\InvestmentNotebook\RESULT\CFFEX_2010-11.zip ...
正在解压缩 d:\Development\Jupyter\InvestmentNotebook\RESULT\CFFEX_2010-12.zip ...
正在解压缩 d:\Development\Jupyter\InvestmentNotebook\RESULT\CFFEX_2011-01.zip ...
正在解压缩 d:\Development\Jupyter\InvestmentNotebook\RESULT\CFFEX_2011-02.zip ...
正在解压缩 d:\Development\Jupyter\InvestmentNotebook\RESULT\CFFEX_2011-03.zip ...
正在解压缩 d:\Development\Jupyter\InvestmentNotebook\RESULT\CFFEX_2011-04.zip ...

FileExistsError: [WinError 183] 当文件已存在时，无法创建该文件。: 'd:\\Development\\Jupyter\\InvestmentNotebook\\RESULT\\temp2\\╫╪Θ╡╙═.csv' -> 'd:\\Development\\Jupyter\\InvestmentNotebook\\RESULT\\temp2\\棕榈油.csv'

哦嚯，出错了。

这是我第一次在 Jupyter Notebook 中保留出错信息，以往的可能因为笔误什么的原因导致的错误都直接修改完了。

根据出错的原因和最后两行运行信息，<DCE_棕榈油_2007.zip> 和 <DCE_棕榈油_2008.zip> 两个压缩包中的文件都命名为 <棕榈油.csv>。按照工作顺序，第一个文件（2007 年的）解压出来以乱码名称命名，然后重命名为编码正确的文件名（即 <棕榈油.csv>）之后，第二个文件（2008 年的）解压出来以乱码名称命名，但是不能再重命名了，文件名重复了。于是出错，程序崩溃。

因为 Python 的 Path 不保证每次枚举条目的时候的顺序，所以可能在本地运行的时候出错的文件有所不同。但原理是一样的，这可以通过使用 WinZip、WinRAR、7-Zip 等等压缩解压软件打开大商所相同品种不同年份的压缩文件查看。

大商所还真的是麻烦。

解决的办法有两个：

- 解压到不同目录中；
- 解压后的文件添加年份来区别。

如何判断哪一个办法比较好？我们不妨考虑下一步工作的要求。下一步应该是读取文件内容，整理后保存为文件或者写入数据库。那么，第一个办法需要在子程序间传递文件列表（列表可能有点大），或者再次搜索文件夹（这次是嵌套文件夹了，因为大商所不全是压缩文件）；第二个办法似乎看起来没什么不好的，解压出来的结果干干净净。

OK，按照第二个办法干。

In [6]:
# 再换一个解压路径，我没辙了，变量名加数字吧。果然编程最大的问题就是起名字。
unzip_path_3: Path = PATH_FOR_RESULT.joinpath('temp3')

# 如果新的解压路径不存在，创建它。
if not unzip_path_3.exists():
    unzip_path_3.mkdir()

# 解压
for item in search_all_zip_file(PATH_FOR_RESULT):
    print(f'正在解压缩 {item} ...')
    
    # 获得新解压出来的文件列表。
    unzipped_list = unzip_file(
        zipped_file=item,
        unzip_path=unzip_path_3,
        keep=True
    )
    
    # 如果 <item> 以 <DCE> 开头，那么重命名新解压的文件。
    if item.name[:3] == 'DCE':
        unzipped_list[0].rename(
            unzip_path_3.joinpath(f'{item.stem}{unzipped_list[0].suffix}')
        )

正在解压缩 d:\Development\Jupyter\InvestmentNotebook\RESULT\CFFEX_2010-04.zip ...
正在解压缩 d:\Development\Jupyter\InvestmentNotebook\RESULT\CFFEX_2010-05.zip ...
正在解压缩 d:\Development\Jupyter\InvestmentNotebook\RESULT\CFFEX_2010-06.zip ...
正在解压缩 d:\Development\Jupyter\InvestmentNotebook\RESULT\CFFEX_2010-07.zip ...
正在解压缩 d:\Development\Jupyter\InvestmentNotebook\RESULT\CFFEX_2010-08.zip ...
正在解压缩 d:\Development\Jupyter\InvestmentNotebook\RESULT\CFFEX_2010-09.zip ...
正在解压缩 d:\Development\Jupyter\InvestmentNotebook\RESULT\CFFEX_2010-10.zip ...
正在解压缩 d:\Development\Jupyter\InvestmentNotebook\RESULT\CFFEX_2010-11.zip ...
正在解压缩 d:\Development\Jupyter\InvestmentNotebook\RESULT\CFFEX_2010-12.zip ...
正在解压缩 d:\Development\Jupyter\InvestmentNotebook\RESULT\CFFEX_2011-01.zip ...
正在解压缩 d:\Development\Jupyter\InvestmentNotebook\RESULT\CFFEX_2011-02.zip ...
正在解压缩 d:\Development\Jupyter\InvestmentNotebook\RESULT\CFFEX_2011-03.zip ...
正在解压缩 d:\Development\Jupyter\InvestmentNotebook\RESULT\CFFEX_2011-04.zip ...

FileExistsError: [WinError 183] 当文件已存在时，无法创建该文件。: 'd:\\Development\\Jupyter\\InvestmentNotebook\\RESULT\\temp3\\╦∙─┌║╧╘╝╨╨╟Θ▒¿▒φ.xls' -> 'd:\\Development\\Jupyter\\InvestmentNotebook\\RESULT\\temp3\\所内合约行情报表.xls'

哦嚯，上期所也出现这种情况。

仔细观察上期所文件，2019 年开始，上期所压缩包中有多个文件了，必然有命名上的区别。所以是压缩包中只有一个文件的需要重命名。

再修改一下。

In [7]:
# 第4个解压路径……
unzip_path_4: Path = PATH_FOR_RESULT.joinpath('temp4')

# 如果新的解压路径不存在，创建它。
if not unzip_path_4.exists():
    unzip_path_4.mkdir()

# 解压
for item in search_all_zip_file(PATH_FOR_RESULT):
    print(f'正在解压缩 {item} ...')
    
    # 获得新解压出来的文件列表。
    unzipped_list = unzip_file(
        zipped_file=item,
        unzip_path=unzip_path_4,
        keep=True
    )
    
    # 如果 <item> 以 <DCE> 开头，那么重命名新解压的文件。
    if item.name[:3] == 'DCE':
        unzipped_list[0].rename(
            unzip_path_4.joinpath(f'{item.stem}{unzipped_list[0].suffix}')
        )
    # 如果 <item> 以 <SHFE> 开头，那么重命名新解压的文件。
    elif item.name[:4] == 'SHFE' and len(unzipped_list) == 1:
        unzipped_list[0].rename(
            unzip_path_4.joinpath(f'{item.stem}{unzipped_list[0].suffix}')
        )

正在解压缩 d:\Development\Jupyter\InvestmentNotebook\RESULT\CFFEX_2010-04.zip ...
正在解压缩 d:\Development\Jupyter\InvestmentNotebook\RESULT\CFFEX_2010-05.zip ...
正在解压缩 d:\Development\Jupyter\InvestmentNotebook\RESULT\CFFEX_2010-06.zip ...
正在解压缩 d:\Development\Jupyter\InvestmentNotebook\RESULT\CFFEX_2010-07.zip ...
正在解压缩 d:\Development\Jupyter\InvestmentNotebook\RESULT\CFFEX_2010-08.zip ...
正在解压缩 d:\Development\Jupyter\InvestmentNotebook\RESULT\CFFEX_2010-09.zip ...
正在解压缩 d:\Development\Jupyter\InvestmentNotebook\RESULT\CFFEX_2010-10.zip ...
正在解压缩 d:\Development\Jupyter\InvestmentNotebook\RESULT\CFFEX_2010-11.zip ...
正在解压缩 d:\Development\Jupyter\InvestmentNotebook\RESULT\CFFEX_2010-12.zip ...
正在解压缩 d:\Development\Jupyter\InvestmentNotebook\RESULT\CFFEX_2011-01.zip ...
正在解压缩 d:\Development\Jupyter\InvestmentNotebook\RESULT\CFFEX_2011-02.zip ...
正在解压缩 d:\Development\Jupyter\InvestmentNotebook\RESULT\CFFEX_2011-03.zip ...
正在解压缩 d:\Development\Jupyter\InvestmentNotebook\RESULT\CFFEX_2011-04.zip ...

OK，完成了。

其实还有第三个办法，就是解压一个压缩文件，立刻处理新解压出来的数据文件，处理完了就删除数据文件。这样也就不会出现命名重复的情况了。

这个办法在下一部分进行吧。